In [1]:
import graph_tool.all as gt

g = gt.collection.ns["eu_airlines"]

In [2]:
latitude = g.vp["nodeLat"]
longitude = g.vp["nodeLong"]
name = g.vp["name"]
layer = g.ep["layer"]
SELECTED = "EPWA"

from_warsaw = []

for e in g.edges():
    source = g.vertex(e.source())
    target = g.vertex(e.target())

    if name[e.source()] == SELECTED:
        from_warsaw.append(target)
    elif name[e.target()] == SELECTED:
        from_warsaw.append(source)

In [12]:
import folium
import matplotlib.pyplot as plt

map = folium.Map(location=[50.0, 10.0], zoom_start=4)

cmap = plt.get_cmap("gist_ncar")
norm = plt.Normalize(min(layer), max(layer))

for v in g.vertices():
    lat, lon = latitude[v], longitude[v]
    if name[v] == "EPWA":
        folium.Marker([lat, lon], popup=name[v]).add_to(map)
    else:
        color = "red" if v in from_warsaw else "blue"
        radius = 4 if v in from_warsaw else 2
        folium.CircleMarker([lat, lon], popup=name[v], color=color, radius=radius).add_to(map)

for e in g.edges():
    source = g.vertex(e.source())
    target = g.vertex(e.target())

    if not (name[e.source()] == SELECTED or name[e.target()] == SELECTED):
        continue

    edge_layer = layer[e]
    color = cmap(norm(edge_layer))
    color = "#{:02x}{:02x}{:02x}".format(
        int(color[0] * 255), int(color[1] * 255), int(color[2] * 255)
    )
    folium.PolyLine(
        [(latitude[source], longitude[source]), (latitude[target], longitude[target])],
        color=color,
        popup=f"{name[e.source()]} - {name[e.target()]}, by {layer[e]}",
    ).add_to(map)

title_html = """
    <h3 align="center" style="font-size:16px"><b>Interaktywna Mapa Połączeń Lotniczych w Europie</b></h3>
    <p align="center" style="font-size:12px">Mapa przedstawiająca połączenia lotnicze pomiędzy EPWA a innymi lotniskami w Europie</p>
"""
map.get_root().html.add_child(folium.Element(title_html))

map.save("map.html")